Kaggle Compitition- Contradictory, My Dear Watson. 
Comparing two setences via tokenize each pair of sentences and doing word embedding over them.
Rank - 12 
Accuracy =0.92

In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


#inporting files from the folder
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing bert tokenizer for pretrained model
!pip install transformers
from transformers import BertTokenizer, TFBertModel, TFAutoModel,AutoTokenizer
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
os.environ["WANDB_API_KEY"] = "0" ## to silence warning

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu) 
except ValueError: 
    strategy = tf.distribute.get_strategy() 
    print('Number of replicas:', strategy.num_replicas_in_sync) 

In [ ]:
#reading csv files - train and test 
train = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')
test = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

In [ ]:
#printing first 5 rows 
train.head()

In [ ]:
train.describe(include='all')

In [ ]:
#plotting missingno. bar and matrix to get null values
import missingno as msn
msn.bar(train)
msn.matrix(train)


In [ ]:
# we dont get any null values still reconfirming via isnull() function
train.isnull().sum()

In [ ]:
#dropping language axis as we already have lang_abv to distinguish languages
train=train.drop('language',axis=1)
test=test.drop('language',axis=1)
train.head()


In [ ]:
#countplot for different lang_abv
sns.countplot(x='lang_abv', data=train)


In [ ]:
plt.figure(figsize=(9,9))
train.groupby('lang_abv').size().plot(kind='pie', autopct='%1.1f%%')

In [ ]:
#Checking class distribution for particular labels
train.label.value_counts().plot(kind='bar')
plt.xlabel("Classes")
plt.ylabel("no. of occurences")
plt.title("Train Classes distribution")
plt.show()

In [ ]:
#distribution of languages and labels
import seaborn as sns

fig, ax = plt.subplots(figsize = (12,5))

#for maximum aesthetics
palette = sns.cubehelix_palette(8, start=2, rot=0, dark=0, light=.95, reverse=True)

graph1 = sns.countplot(train['lang_abv'], hue = train['label'])#, palette = palette)

#set title
graph1.set_title('Distribution of Languages and Labels')

plt.tight_layout()
plt.show()

In [ ]:
# Calling autotokenizer for pretrained model- "joeddav/xlm-roberta-large-xnli"
from transformers import BertTokenizer, TFBertModel, TFAutoModel,AutoTokenizer

#This model takes xlm-roberta-large and fine-tunes it on a combination of NLI data in 15 languages.
#tokenizer = BertTokenizer.from_pretrained(model_name) 

model_name = "joeddav/xlm-roberta-large-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# A function to encode our premise sentences into tokens (word embedding)
def encode_premise_sentence(s):
    tokens = []
    tokens.append('[CLS]')
    tokens+=list(tokenizer.tokenize(s))
    return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
# A function to encode our hypothesis sentences into tokens (word embedding)
def encode_hypothesis_sentence(s):
    tokens = []
    tokens.append('[SEP]')
    #[SEP] is for separating sentences for the next sentence prediction task
    tokens+=list(tokenizer.tokenize(s))
    tokens.append('[SEP]')
    return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
#Tokenizing the premise and hypotheis sentences  for len(train) i.e #(rows)
tokenized=[]
for each in range(len(train)):
    pre=encode_premise_sentence(train['premise'][each])
    hyp=encode_hypothesis_sentence(train['hypothesis'][each])
    #Comibining both tokenized sentences to one 
    tokenized.append(pre+hyp)
train['tokenized'] = tokenized
train.head(5)

In [ ]:
#adding padding in the tokenizer
mask=[]
for each in range(len(train)):
    padded_sequences = tokenizer(train['premise'][each],train['hypothesis'][each], padding=True,add_special_tokens = True)
    mask.append(padded_sequences)
train['masked'] = mask
train.head(5)

In [ ]:
#Building tf model from pretrained joeddav/xlm-roberta-large-xnli
#Appylying bert_encoder and optimizers
max_len=237 

def build_model():
    bert_encoder = TFAutoModel.from_pretrained('joeddav/xlm-roberta-large-xnli')
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    #input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")
    
    embedding = bert_encoder([input_word_ids, input_mask])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
def input_convert(data):
    inputs   = {
            'input_word_ids' :[],
            'input_mask'     :[]  }
    for each in data:
        inputs['input_word_ids'].append(each['input_ids'])
        inputs['input_mask'].append(each['attention_mask'])
        
    
    inputs['input_word_ids']= tf.ragged.constant( inputs['input_word_ids']).to_tensor()
    inputs['input_mask']= tf.ragged.constant( inputs['input_mask']).to_tensor()
  
    
    return inputs

In [ ]:
train_input= input_convert(train['masked'].values)
for key in train_input.keys():
    train_input[key] = train_input[key][:,:max_len]    

In [ ]:
#appylying the build_model and fitting on train data
early_stop = tf.keras.callbacks.EarlyStopping(patience=3,restore_best_weights=True)
with strategy.scope():
    model = build_model()
    model.summary()
    model.fit(train_input, train['label'].values, epochs = 5, verbose = 1, batch_size = 128, validation_split = 0.1 ,callbacks=[early_stop])

In [ ]:
#applying tokenizer on test data
mask=[]
for each in range(len(test)):
    padded_sequences = tokenizer(test['premise'][each],test['hypothesis'][each], padding=True,add_special_tokens = True)
    mask.append(padded_sequences)
test['masked'] = mask
test.head(5)

In [ ]:
test_input= input_convert(test['masked'].values)
for key in test_input.keys():
    test_input[key] = test_input[key][:,:max_len]

In [ ]:
predictions = [np.argmax(i) for i in model.predict(test_input)]

In [ ]:
test

In [ ]:
#creating a new file 'Submission'
submission = test['id'].copy().to_frame()
submission['prediction'] = predictions
submission.to_csv('/kaggle/submission.csv', index=False)

In [ ]:
submission

In [ ]:
submission.to_csv("submission.csv", index = False)